In [6]:
import xarray as xr
from pathlib import Path
import pandas as pd

In [8]:
BASE_PATH = Path("/mnt/storage_ssd_4tb/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/prepared_ML_training_data/v15/train")
DATA_SOURCE_NAMES = ("gsp", "hrvsatellite", "nwp", "pv", "satellite", "sun", "topographic")

In [7]:
def load_and_check_batch(filename: Path) -> pd.Series:
    """Loads a batch NetCDF file. Computes stats. Returns pd.Series mapping stat name to stat value."""
    dataset = xr.load_dataset(filename, mode="r")
    

In [ ]:
def main():
    # Use multiprocessing.Pool.map to load load_and_check_batch for each filename.

In [ ]:
%%time
filename = BASE_PATH / "nwp" / "000000.nc"
dataset = xr.load_dataset(filename, mode="r")